# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS-109A Introduction to Data Science 


## Fine Particulate Air Pollution and COVID-19

**Harvard University**<br>
**Spring 2020**<br>
Jack Luby, Hakeem Angulu, and Louie Ayre <br>

---



In [1]:
## Set formatting to CS109 standard
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

In [2]:
# The classics
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import statsmodels.api as sm

In [3]:
# load main df
main_df = pd.read_csv('data/main.csv')
main_df.head()

,fips,pm25,poverty,pop_density,med_house_value,pct_blk,med_household_income,pct_owner_occ,pct_hispanic,education,...,liquor_stores_open,firearms_sellers_open,days_since_closing_restaurants_except_takeout,days_since_closing_gyms,days_since_closing_movie_theaters,froze_evictions,order_freezing_utility_shutoffs,infection_rate,death_rate,death_rate_amongst_infected
0,1001,0.671200,0.247670,0.001286,0.117213,0.191395,0.332512,0.748946,0.027681,0.305812,...,1,1,0.928571,0.781818,0.781818,0,0,0.000960,0.000054,0.056604
1,1003,0.586507,0.156516,0.001814,0.160143,0.094970,0.308184,0.736193,0.044943,0.263580,...,1,1,0.928571,0.781818,0.781818,0,0,0.000908,0.000024,0.026455
2,1005,0.633035,0.503097,0.000401,0.060861,0.475758,0.120589,0.613978,0.042898,0.552148,...,1,1,0.928571,0.781818,0.781818,0,0,0.001823,0.000039,0.021277
3,1007,0.687609,0.221327,0.000499,0.064549,0.222755,0.216678,0.750731,0.024282,0.398816,...,1,1,0.928571,0.781818,0.781818,0,0,0.001909,0.000000,0.000000
4,1009,0.643753,0.229143,0.001238,0.093443,0.014954,0.245581,0.786262,0.091266,0.451338,...,1,1,0.928571,0.781818,0.781818,0,0,0.000694,0.000000,0.000000


In [4]:
# establish train and test sets 
responses = ['infection_rate', 'death_rate', 'death_rate_amongst_infected']

X_train, X_test, y_train, y_test = train_test_split(main_df.drop(responses + ['fips'], axis=1),
                                                    main_df[responses], test_size=0.2,
                                                    random_state=109)

In [5]:
# add constants
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

# initialize lists
models = []
train_scores = []
test_scores = []

# for each response variable:
for i in range(len(responses)):
    
    # fit model
    model = sm.OLS(y_train[responses[i]], X_train_const).fit()
    models.append(model)
    
    # predict on train and test
    train_predict = model.predict(X_train_const)
    test_predict = model.predict(X_test_const)

    # get mse
    train_score = metrics.mean_squared_error(y_train[responses[i]], train_predict)
    test_score = metrics.mean_squared_error(y_test[responses[i]], test_predict)
    
    train_scores.append(train_score)
    test_scores.append(test_score)

In [6]:
models[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         infection_rate   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.244
Method:                 Least Squares   F-statistic:                     29.21
Date:                Sun, 10 May 2020   Prob (F-statistic):          5.35e-122
Time:                        16:29:23   Log-Likelihood:                 9014.4
No. Observations:                2272   AIC:                        -1.797e+04
Df Residuals:                    2245   BIC:                        -1.782e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                         -4.378e-05      0.004     -0.012      0.990      -0.007       0.007
pm25                                             -0.0010      0.001     -1.218      0.223      -0.003       0.001
poverty                                          -0.0001      0.002     -0.062      0.950      -0.004       0.004
pop_density                                       0.0955      0.005     20.593      0.000       0.086       0.105
med_house_value                                  -0.0002      0.002     -0.098      0.922      -0.005       0.004
pct_blk                                          -0.0012      0.004     -0.345      0.730      -0.008       0.006
med_household_income                              0.0070      0.002      3.285      0.001       0.003       0.011
pct_owner_occ                                     0.0006      0.002      0.352      0.725      -0.003       0.004
pct_hispanic                                     -0.0010      0.001     -0.820      0.412      -0.003       0.001
education                                         0.0064      0.001      5.124      0.000       0.004       0.009
pct_asian                                        -0.0143      0.007     -1.991      0.047      -0.028      -0.000
pct_native                                       -0.0044      0.004     -1.090      0.276      -0.012       0.004
pct_white                                        -0.0064      0.003     -1.880      0.060      -0.013       0.000
days_since_state_of_emergency                 -9.987e-05      0.001     -0.127      0.899      -0.002       0.001
days_since_closing_K-12                           0.0022      0.001      2.306      0.021       0.000       0.004
days_since_closing_day_cares                   4.641e-05      0.000      0.133      0.894      -0.001       0.001
days_since_banning_visitors_to_nursing_homes      0.0005      0.000      1.629      0.103    -9.8e-05       0.001
days_since_shelter_in_place                       0.0006      0.000      1.607      0.108      -0.000       0.001
days_since_closing_non-essential_businesses    9.706e-05      0.000      0.227      0.820      -0.001       0.001
religious_gatherings_exempt                      -0.0006      0.000     -1.858      0.063      -0.001    3.07e-05
liquor_stores_open                                0.0010      0.001      1.195      0.232      -0.001       0.003
firearms_sellers_open                             0.0002      0.000      0.432      0.665      -0.001       0.001
days_since_closing_restaurants_except_takeout    -0.0029      0.002     -1.826      0.068      -0.006       0.000
days_since_closing_gyms                           0.0022      0.002      1.352      0.176      -0.001       0.005
days_

In [7]:
models[1].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             death_rate   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.640
Method:                 Least Squares   F-statistic:                     156.3
Date:                Sun, 10 May 2020   Prob (F-statistic):               0.00
Time:                        16:29:25   Log-Likelihood:                 16327.
No. Observations:                2272   AIC:                        -3.260e+04
Df Residuals:                    2245   BIC:                        -3.245e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                            -0.0006      0.000     -4.292      0.000      -0.001      -0.000
pm25                                          -2.032e-05   3.27e-05     -0.621      0.535   -8.45e-05    4.39e-05
poverty                                           0.0003   7.39e-05      3.483      0.001       0.000       0.000
pop_density                                       0.0102      0.000     55.116      0.000       0.010       0.011
med_house_value                                3.526e-05   8.99e-05      0.392      0.695      -0.000       0.000
pct_blk                                           0.0004      0.000      2.924      0.003       0.000       0.001
med_household_income                              0.0003   8.48e-05      3.738      0.000       0.000       0.000
pct_owner_occ                                     0.0003   6.97e-05      4.191      0.000       0.000       0.000
pct_hispanic                                   4.817e-05   5.02e-05      0.959      0.338   -5.03e-05       0.000
education                                      4.808e-05   4.99e-05      0.963      0.336   -4.98e-05       0.000
pct_asian                                        -0.0012      0.000     -4.345      0.000      -0.002      -0.001
pct_native                                        0.0001      0.000      0.761      0.447      -0.000       0.000
pct_white                                         0.0001      0.000      0.989      0.323      -0.000       0.000
days_since_state_of_emergency                  6.738e-06   3.15e-05      0.214      0.831   -5.51e-05    6.86e-05
days_since_closing_K-12                           0.0001   3.82e-05      3.198      0.001    4.72e-05       0.000
days_since_closing_day_cares                  -5.946e-06    1.4e-05     -0.426      0.670   -3.33e-05    2.15e-05
days_since_banning_visitors_to_nursing_homes   5.902e-05   1.18e-05      4.996      0.000    3.59e-05    8.22e-05
days_since_shelter_in_place                    2.231e-05    1.4e-05      1.594      0.111   -5.13e-06    4.98e-05
days_since_closing_non-essential_businesses    5.605e-05   1.71e-05      3.282      0.001    2.26e-05    8.96e-05
religious_gatherings_exempt                   -6.639e-05   1.19e-05     -5.560      0.000   -8.98e-05    -4.3e-05
liquor_stores_open                             6.113e-05   3.45e-05      1.771      0.077   -6.57e-06       0.000
firearms_sellers_open                         -9.817e-06   1.43e-05     -0.687      0.492   -3.78e-05    1.82e-05
days_since_closing_restaurants_except_takeout    -0.0003   6.37e-05     -4.655      0.000      -0.000      -0.000
days_since_closing_gyms                           0.0002   6.63e-05      2.686      0.007    4.81e-05       0.000
days_

In [8]:
models[2].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     death_rate_amongst_infected   R-squared:                       0.025
Model:                                     OLS   Adj. R-squared:                  0.014
Method:                          Least Squares   F-statistic:                     2.224
Date:                         Sun, 10 May 2020   Prob (F-statistic):           0.000373
Time:                                 16:29:26   Log-Likelihood:                 2940.1
No. Observations:                         2272   AIC:                            -5826.
Df Residuals:                             2245   BIC:                            -5672.
Df Model:                                   26                                         
Covariance Type:                     nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                            -0.0583      0.053     -1.109      0.267      -0.161       0.045
pm25                                              0.0258      0.012      2.172      0.030       0.002       0.049
poverty                                           0.0522      0.027      1.951      0.051      -0.000       0.105
pop_density                                       0.0697      0.067      1.037      0.300      -0.062       0.202
med_house_value                                  -0.0049      0.033     -0.150      0.881      -0.069       0.059
pct_blk                                           0.0420      0.051      0.818      0.414      -0.059       0.143
med_household_income                              0.0142      0.031      0.462      0.644      -0.046       0.074
pct_owner_occ                                     0.0441      0.025      1.749      0.080      -0.005       0.094
pct_hispanic                                      0.0107      0.018      0.590      0.555      -0.025       0.046
education                                        -0.0342      0.018     -1.891      0.059      -0.070       0.001
pct_asian                                         0.1054      0.104      1.013      0.311      -0.099       0.309
pct_native                                        0.0172      0.059      0.291      0.771      -0.098       0.133
pct_white                                         0.0402      0.049      0.819      0.413      -0.056       0.136
days_since_state_of_emergency                     0.0085      0.011      0.743      0.458      -0.014       0.031
days_since_closing_K-12                           0.0060      0.014      0.434      0.665      -0.021       0.033
days_since_closing_day_cares                     -0.0019      0.005     -0.385      0.700      -0.012       0.008
days_since_banning_visitors_to_nursing_homes      0.0151      0.004      3.526      0.000       0.007       0.023
days_since_shelter_in_place                       0.0024      0.005      0.482      0.630      -0.007       0.012
days_since_closing_non-essential_businesses       0.0090      0.006      1.454      0.146      -0.003       0.021
religious_gatherings_exempt                      -0.0099      0.004     -2.291      0.022      -0.018      -0.001
liquor_stores_open                               -0.0021      0.013     -0.169      0.866      -0.027       0.022
firearms_sellers_open                            -0.0091      0.005     -1.766      0.078      -0.019       0.001
days_since_closing_restaurants_except_takeout    -0.0229      0.023     -0.994      0.320      -0.068       0.022
days_since_closing_g

In [9]:
# summarize mse
df = pd.DataFrame(responses, columns = ['Response'])
df['Training MSE'] = train_scores
df['Testing MSE'] = test_scores
df

,Response,Training MSE,Testing MSE
0,infection_rate,2.095594e-05,9.976492e-06
1,death_rate,3.354571e-08,3.980368e-08
2,death_rate_amongst_infected,4.400551e-03,3.681010e-03


In [10]:
# statsmodel does not support feature selection. The following source was adapted to do so here: https://planspace.org/20150423-forward_selection_with_statsmodels/

def forward_feature_selection(X, y):
    '''
    Forward feature selection: iteratively add the feature the raises 
    adjusted r-squared the most, untilit cannot be raised anymore or
    there are no more features
    
    X: predictors
    y: response
    
    '''
    
    sel = []
    cols = set(X.columns)
    
    # initialize current adjusted r^2 value and new value
    r, new_r = 0.0, 0.0
    
    # iterate through predictors
    while cols and r == new_r:
        scores = []
        
        # calculate new score resulting from adding each remaing predictor
        for col in cols:
            score = sm.OLS(y, sm.add_constant(X[sel+[col]])).fit().rsquared_adj
            scores.append((score, col))
        scores.sort()
        
        # take the best 
        new_r, best_col = scores.pop()
        
        # if improved, select new predictor and continue
        if r < new_r:
            cols.remove(best_col)
            sel.append(best_col)
            r = new_r
    
    # create model
    model = sm.OLS(y, sm.add_constant(X[sel])).fit()
    return model

In [11]:
# initialize lists
ffs_models = []
ffs_train_scores = []
ffs_test_scores = []

# for each response:
for i in range(len(responses)):
    
    # fit model
    ffs_model = forward_feature_selection(X_train, y_train[responses[i]])
    ffs_models.append(ffs_model)
    
    # predict on train and test
    ffs_train_predict = ffs_model.predict(X_train_const[list(ffs_model.params.index)])
    ffs_test_predict = ffs_model.predict(X_test_const[list(ffs_model.params.index)])

    # get mse
    ffs_train_score = metrics.mean_squared_error(y_train[responses[i]], ffs_train_predict)
    ffs_test_score = metrics.mean_squared_error(y_test[responses[i]], ffs_test_predict)
    
    ffs_train_scores.append(ffs_train_score)
    ffs_test_scores.append(ffs_test_score)

In [13]:
ffs_models[0].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         infection_rate   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.247
Method:                 Least Squares   F-statistic:                     42.29
Date:                Sun, 10 May 2020   Prob (F-statistic):          4.40e-128
Time:                        16:33:49   Log-Likelihood:                 9014.0
No. Observations:                2272   AIC:                        -1.799e+04
Df Residuals:                    2253   BIC:                        -1.788e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                            -0.0011      0.001     -0.737      0.461      -0.004       0.002
pop_density                                       0.0950      0.004     21.965      0.000       0.087       0.103
pct_white                                        -0.0049      0.001     -6.111      0.000      -0.007      -0.003
education                                         0.0065      0.001      5.770      0.000       0.004       0.009
med_household_income                              0.0071      0.001      5.982      0.000       0.005       0.009
days_since_closing_movie_theaters                 0.0004      0.001      0.579      0.563      -0.001       0.002
order_freezing_utility_shutoffs               -1.919e-05   4.24e-06     -4.532      0.000   -2.75e-05   -1.09e-05
pct_asian                                        -0.0139      0.006     -2.528      0.012      -0.025      -0.003
froze_evictions                                2.281e-05   5.59e-06      4.081      0.000    1.18e-05    3.38e-05
days_since_closing_K-12                           0.0023      0.001      2.720      0.007       0.001       0.004
pct_hispanic                                     -0.0009      0.001     -0.907      0.364      -0.003       0.001
pct_native                                       -0.0031      0.002     -1.650      0.099      -0.007       0.001
pm25                                             -0.0009      0.001     -1.218      0.223      -0.002       0.001
days_since_banning_visitors_to_nursing_homes      0.0005      0.000      1.869      0.062   -2.58e-05       0.001
religious_gatherings_exempt                      -0.0006      0.000     -2.278      0.023      -0.001   -8.13e-05
days_since_closing_restaurants_except_takeout    -0.0028      0.001     -1.921      0.055      -0.006    5.81e-05
days_since_shelter_in_place                       0.0005      0.000      1.645      0.100   -9.55e-05       0.001
days_since_closing_gyms                           0.0021      0.002      1.328      0.184      -0.001       0.005
liquor_stores_open                                0.0011      0.001      1.282      0.200      -0.001       0.003
==============================================================================
Omnibus:                     4478.232   Durbin-Watson:                   2.040
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         14333142.625
Skew:                          15.185   Prob(JB):                         0.00
Kurtosis:                     390.923   Cond. No.                     2.75e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified

In [14]:
ffs_models[1].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             death_rate   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.641
Method:                 Least Squares   F-statistic:                     214.1
Date:                Sun, 10 May 2020   Prob (F-statistic):               0.00
Time:                        16:33:53   Log-Likelihood:                 16326.
No. Observations:                2272   AIC:                        -3.261e+04
Df Residuals:                    2252   BIC:                        -3.250e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                            -0.0005   7.21e-05     -6.749      0.000      -0.001      -0.000
pop_density                                       0.0102      0.000     55.708      0.000       0.010       0.011
pct_blk                                           0.0003   3.52e-05      7.592      0.000       0.000       0.000
pct_owner_occ                                     0.0003   6.35e-05      4.549      0.000       0.000       0.000
pct_asian                                        -0.0014      0.000     -6.118      0.000      -0.002      -0.001
days_since_closing_non-essential_businesses     4.88e-05   1.54e-05      3.170      0.002    1.86e-05     7.9e-05
order_freezing_utility_shutoffs               -7.899e-07   1.69e-07     -4.682      0.000   -1.12e-06   -4.59e-07
days_since_banning_visitors_to_nursing_homes   5.801e-05   1.13e-05      5.140      0.000    3.59e-05    8.01e-05
religious_gatherings_exempt                    -6.21e-05   9.95e-06     -6.244      0.000   -8.16e-05   -4.26e-05
froze_evictions                                1.159e-06   2.33e-07      4.967      0.000    7.01e-07    1.62e-06
days_since_closing_restaurants_except_takeout    -0.0003   6.27e-05     -4.683      0.000      -0.000      -0.000
days_since_closing_movie_theaters              3.024e-05   2.97e-05      1.019      0.308    -2.8e-05    8.85e-05
days_since_closing_K-12                           0.0001   3.49e-05      3.229      0.001    4.42e-05       0.000
days_since_closing_gyms                           0.0002   6.56e-05      2.776      0.006    5.34e-05       0.000
med_house_value                                5.595e-05   8.48e-05      0.660      0.509      -0.000       0.000
poverty                                           0.0003   6.86e-05      3.756      0.000       0.000       0.000
med_household_income                              0.0003   7.98e-05      3.898      0.000       0.000       0.000
days_since_shelter_in_place                    2.403e-05   1.25e-05      1.918      0.055   -5.44e-07    4.86e-05
liquor_stores_open                             4.471e-05   3.12e-05      1.431      0.153   -1.66e-05       0.000
education                                      5.417e-05   3.92e-05      1.381      0.167   -2.28e-05       0.000
==============================================================================
Omnibus:                     1577.646   Durbin-Watson:                   1.905
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           170076.985
Skew:                           2.435   Prob(JB):                         0.00
Kurtosis:                      45.106   Cond. No.                     2.86e+03
==================================================================

In [15]:
ffs_models[2].summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:     death_rate_amongst_infected   R-squared:                       0.023
Model:                                     OLS   Adj. R-squared:                  0.018
Method:                          Least Squares   F-statistic:                     4.157
Date:                         Sun, 10 May 2020   Prob (F-statistic):           7.28e-07
Time:                                 16:33:57   Log-Likelihood:                 2938.1
No. Observations:                         2272   AIC:                            -5848.
Df Residuals:                             2258   BIC:                            -5768.
Df Model:                                   13                                         
Covariance Type:                     nonrobust                                         
=================================================================================================================
                                                    coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------
const                                            -0.0118      0.019     -0.611      0.541      -0.050       0.026
days_since_banning_visitors_to_nursing_homes      0.0148      0.004      3.962      0.000       0.007       0.022
days_since_state_of_emergency                     0.0087      0.008      1.085      0.278      -0.007       0.024
pm25                                              0.0273      0.010      2.753      0.006       0.008       0.047
days_since_closing_movie_theaters                 0.0109      0.009      1.196      0.232      -0.007       0.029
firearms_sellers_open                            -0.0089      0.004     -2.079      0.038      -0.017      -0.001
religious_gatherings_exempt                      -0.0075      0.003     -2.199      0.028      -0.014      -0.001
days_since_closing_non-essential_businesses       0.0083      0.005      1.700      0.089      -0.001       0.018
days_since_closing_restaurants_except_takeout    -0.0140      0.010     -1.363      0.173      -0.034       0.006
education                                        -0.0311      0.014     -2.162      0.031      -0.059      -0.003
poverty                                           0.0405      0.017      2.409      0.016       0.008       0.074
pct_owner_occ                                     0.0441      0.022      1.978      0.048       0.000       0.088
pct_asian                                         0.0739      0.070      1.062      0.288      -0.063       0.210
pop_density                                       0.0664      0.064      1.043      0.297      -0.058       0.191
==============================================================================
Omnibus:                     2797.267   Durbin-Watson:                   1.944
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           495337.375
Skew:                           6.428   Prob(JB):                         0.00
Kurtosis:                      74.184   Cond. No.                         116.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# summarize mse

ffs_df = pd.DataFrame(responses, columns = ['Response'])
ffs_df['Training MSE'] = ffs_train_scores
ffs_df['Testing MSE'] = ffs_test_scores
ffs_df

,Response,Training MSE,Testing MSE
0,infection_rate,2.096326e-05,9.995124e-06
1,death_rate,3.358671e-08,3.994532e-08
2,death_rate_amongst_infected,4.408359e-03,3.695581e-03
